### Import packages needed

In [1]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds

### Data

In [32]:

mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']


num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples

num_validation_samples = tf.cast(num_validation_samples, tf.int64)


num_test_samples = mnist_info.splits['test'].num_examples

num_test_samples = tf.cast(num_test_samples, tf.int64)



def scale(image, label):
    
    image = tf.cast(image, tf.float32)
    
    image /= 255.

    return image, label



scaled_train_and_validation_data = mnist_train.map(scale)


test_data = mnist_test.map(scale)




BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)


validation_data = shuffled_train_and_validation_data.take(num_validation_samples)


train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100


train_data = train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samples)


test_data = test_data.batch(num_test_samples)


validation_inputs, validation_targets = next(iter(validation_data))

### Outline the model

In [36]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # input layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])

### Optimizer and loss function

In [37]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

### Training

In [38]:
NUM_EPOCHS=5

model.fit(train_data,epochs=NUM_EPOCHS,validation_data=(validation_inputs,validation_targets),verbose=2)

Epoch 1/5
540/540 - 17s - loss: 0.3338 - accuracy: 0.9039 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
540/540 - 16s - loss: 0.1392 - accuracy: 0.9597 - val_loss: 0.1237 - val_accuracy: 0.9640
Epoch 3/5
540/540 - 18s - loss: 0.0987 - accuracy: 0.9702 - val_loss: 0.0954 - val_accuracy: 0.9712
Epoch 4/5
540/540 - 19s - loss: 0.0769 - accuracy: 0.9774 - val_loss: 0.0776 - val_accuracy: 0.9777
Epoch 5/5
540/540 - 15s - loss: 0.0625 - accuracy: 0.9807 - val_loss: 0.0681 - val_accuracy: 0.9788


### Test

In [39]:
test_loss,test_accuracy=model.evaluate(test_data)

      1/Unknown - 4s 4s/step - loss: 0.0833 - accuracy: 0.97 - 4s 4s/step - loss: 0.0833 - accuracy: 0.9736

In [40]:
print('Test Loss:{0:.2f}. Test Accuracy:{1:0.2f}%'.format(test_loss,test_accuracy*100.))

Test Loss:0.08. Test Accuracy:97.36%
